In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle

import SALib.analyze.morris
from SALib.sample import morris

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')

from tqdm import tqdm

from infrasim.optimise import *
from infrasim.utils import *

mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['font.family']      = 'Arial'

In [2]:
def process_morris_results(problem,param_values,collected_results,num_levels):
    '''Process output from Morris sensitivitys
    '''
    Si = SALib.analyze.morris.analyze(problem,
                                      np.array(param_values),
                                      np.array(collected_results),
                                      print_to_console=False,
                                      num_levels=num_levels)

    sensitivity_results = pd.DataFrame.from_dict(Si)
    return sensitivity_results


def calculate_relative_influence(sensitivity_results):
    '''Return relative influence (%) of each parameter from sensitivity
    '''
    sensitivity = sensitivity_results.copy()
    sensitivity['rel'] = abs(sensitivity['mu'])/abs(sensitivity['mu']).sum()*100
    sensitivity['mu'] = sensitivity['mu']
    sensitivity = sensitivity[['names','rel']]
    sensitivity = sensitivity.sort_values(by='names',ascending=True).reset_index(drop=True)
    sensitivity.names = sensitivity.names.str.replace('_',' ')
    sensitivity.names = sensitivity.names.str.title()
    sensitivity.names = sensitivity.names.str.replace('Res','RES')
    sensitivity.names = sensitivity.names.str.replace('Coop','COO')
    sensitivity.names = sensitivity.names.str.replace('To','to')
    sensitivity.names = sensitivity.names.str.replace('Westbank','West Bank')
    return sensitivity


def read_results(path='../data/gsa_results/combined_capacities_100_generations.csv'):
    # read results
    combined_caps = pd.read_csv(path)
    # israel
    israel_results = process_morris_results(problem,
                                            param_values,
                                            combined_caps['Israel'].to_list(),
                                            num_levels)
    # jordan
    jordan_results = process_morris_results(problem,
                                            param_values,
                                            combined_caps['Jordan'].to_list(),
                                            num_levels)
    # west bank
    westbank_results = process_morris_results(problem,
                                            param_values,
                                            combined_caps['West Bank'].to_list(),
                                            num_levels)
    # gaza
    gaza_results = process_morris_results(problem,
                                            param_values,
                                            combined_caps['Gaza'].to_list(),
                                            num_levels)
    # return
    return combined_caps,israel_results,jordan_results,westbank_results,gaza_results


def min_max_avg_plot(df,ax=None):
    # line plot
    sns.lineplot(x='value',y='variable',data=df,
                 linewidth=1,zorder=10,color='firebrick',ax=ax)
    # high and low
    sns.scatterplot(x='value',y='variable',data=df[df.stat != 'average'],
                    marker='|',s=100,color='firebrick',zorder=15,ax=ax)
    # mean
    sns.scatterplot(x='value',y='variable',data=df[df.stat == 'average'],
                    marker='o',s=100,color='firebrick',zorder=15,ax=ax)

In [ ]:
#File paths
nodes = '../data/nextra/spatial/network/nodes.shp'
edges = '../data/nextra/spatial/network/edges.shp'
flows = '../data/nextra/nodal_flows/processed_flows_2030.csv'

# params
count       = 1
save_iter   = 25
timesteps   = None
num_levels  = 10
N           = 10

# Set up problem for sensitivity analysis # name:[lower_bound,upper_bound]
params = {'coop_res_target'                 : [0,0.7],
          'jordan_to_westbank'              : [0,10**12],
          'jordan_to_israel'                : [0,10**12],
          'israel_to_westbank'              : [0,10**12],
          'israel_to_jordan'                : [0,10**12],
          'israel_to_gaza'                  : [0,10**12],
          'westbank_to_israel'              : [0,10**12],
          'westbank_to_jordan'              : [0,10**12],
          'self_sufficiency_factor'         : [0,0.9],
         }

problem = {'num_vars'    : len(params.keys()),
           'names'       : [i for i in params.keys()],
           'bounds'      : [params[i] for i in params.keys()]
          }

# create parameter values
param_values = morris.sample(problem,N=N,
                             num_levels=num_levels,
                             local_optimization=True)

# Run analysis with the specified parameter values
collected_caps_israel   = []
collected_caps_jordan   = []
collected_caps_westbank = []
collected_caps_gaza     = []

for param_set in tqdm(param_values,total=len(param_values)):

    model_run = nextra(nodes,edges,flows,
                       timesteps=timesteps,
                       energy_objective=True,
                       scenario='COO',
                       super_sink=False,
                       super_source=False,
                       # params
                       coo_res_factor=param_set[0],
                       jordan_to_westbank=param_set[1],
                       jordan_to_israel=param_set[2],
                       israel_to_westbank=param_set[3],
                       israel_to_jordan=param_set[4],
                       israel_to_gaza=param_set[5],
                       westbank_to_israel=param_set[6],
                       westbank_to_jordan=param_set[7],
                       self_sufficiency_factor=param_set[8],
                      )
    
    # build, run, and get results
    model_run.build()
    model_run.run(pprint=False)
    model_results = model_run.get_results()
    
    # get capacities
    caps = model_results.results_capacities.copy()
    caps = caps.groupby(by='territory').max().reset_index()
    
    # append collected capacities
    collected_caps_israel.append(caps.loc[caps.territory == 'Israel','value'].values[0])
    collected_caps_jordan.append(caps.loc[caps.territory == 'Jordan','value'].values[0])
    collected_caps_westbank.append(caps.loc[caps.territory == 'West Bank','value'].values[0])
    collected_caps_gaza.append(caps.loc[caps.territory == 'Gaza','value'].values[0])
    
    # make dataframes
    combined_caps = pd.DataFrame({'Israel' : collected_caps_israel,
                                  'Jordan' : collected_caps_jordan,
                                  'West Bank' : collected_caps_westbank,
                                  'Gaza' : collected_caps_gaza,})
    
    # save
    if count % save_iter == 0:
        combined_caps.to_csv('../data/gsa_results/combined_capacities_' + str(count) + '_generations.csv',index=False)
    
    # adjust counter
    count = count + 1

  0%|                                                                                                                                                                                                                      | 0/100 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-03



In [ ]:
combined_caps,israel_results,jordan_results,westbank_results,gaza_results = read_results()
collected_caps = combined_caps.copy()

file = open('../outputs/results/model_run_results.pkl','rb')
results = pickle.load(file)
file.close()

baseline_caps = results['COO'].results_capacities.groupby(by='territory').max()['value'].to_dict()

for c in collected_caps.columns:
    collected_caps[c+'_influence'] = (collected_caps[c] - baseline_caps[c])/baseline_caps[c] * 100

collected_caps

isr = calculate_relative_influence(israel_results)
isr['region'] = 'ISR'

jor = calculate_relative_influence(jordan_results)
jor['region'] = 'JOR'

wbk = calculate_relative_influence(westbank_results)
wbk['region'] = 'WBK'

gza = calculate_relative_influence(gaza_results)
gza['region'] = 'GZA'

influence = isr.copy()
influence = influence.append(jor,ignore_index=True)
influence = influence.append(wbk,ignore_index=True)
influence = influence.append(gza,ignore_index=True)
influence['var'] = influence['names']

def custom_lineplot(df,colname,xlims,title,ax):
    x1 = df[colname].min()
    x2 = df[colname].max()
    x3 = df[colname].mean()
    x4 = df[colname].median()
    ax.plot([x1,x2],[0.5,0.5],linestyle='-',color='teal',linewidth=2,zorder=0)
    ax.scatter(x3,0.5,s=250,marker='|',color='teal',linewidth=5)
    ax.scatter(x4,0.5,s=50,marker='o',color='white',linewidth=1,edgecolor='teal')
    ax.set_xlim(xlims)
    ax.set_title(title,loc='left')
    
f = plt.figure(figsize=(10,4))
ax1 = plt.subplot(4,2,1)
ax2 = plt.subplot(4,2,3)
ax3 = plt.subplot(4,2,5)
ax4 = plt.subplot(4,2,7)
ax5 = plt.subplot(1,2,2)

xlims=[-150,300]

custom_lineplot(df=collected_caps,colname='Israel_influence',xlims=xlims,title='ISR',ax=ax1)
custom_lineplot(df=collected_caps,colname='Jordan_influence',xlims=xlims,title='JOR',ax=ax2)
custom_lineplot(df=collected_caps,colname='West Bank_influence',xlims=xlims,title='WBK',ax=ax3)
custom_lineplot(df=collected_caps,colname='Gaza_influence',xlims=xlims,title='GZA',ax=ax4)

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')

ax4.spines['top'].set_visible(False)
ax4.spines['right'].set_visible(False)
ax4.spines['left'].set_visible(False)
ax4.set_yticks([])

ax4.set_xlabel('Change in capacity (%)')

sns.barplot(x='rel',y='var',hue='region',data=influence,ax=ax5,palette='Set2')
ax5.set_ylabel('')
ax5.set_xlabel('Relative Influence (%)')

f.tight_layout() 
f.savefig('../outputs/figures/global_sensitivity.pdf',bbox_inches='tight')